# BIP Framework training example 

In [1]:
using Pkg
Pkg.activate("..")
using BIPs

  Activating project at `~/Documents/WORKS/UBC/RecoBips/BIPs.jl`


In [2]:
using Statistics
using Pkg.Artifacts

Lets begin by bringing in the dataset. It contains tree splits:
* **train**: the training set with 1M jets
* **validation**: the validation set with 400k jets

And of course later we will use the **test** set with other 400k jets to report the results


In [8]:
dataset_path = "/home/jose/Documents/WORKS/UBC/Datalake/reco_ds"

train_data_path = dataset_path*"/train.h5"
val_data_path = dataset_path*"/test.h5";

### Reading the data

In order to read the datasets, we call the `read_dataset` function:
to read the TopQuark format

In [9]:
train_jets, train_labels = BIPs.read_data("RecoTQ", train_data_path)
train_labels = [reinterpret(Bool, b == 1.0) for b in train_labels]
print("Number of entries in the training data: ", length(train_jets))

length(targets[1, :]) = 686539
Number of entries in the training data: 

686539

In [4]:
val_jets, val_labels = BIPs.read_data("RecoTQ", val_data_path)
val_labels = [reinterpret(Bool, b == 1.0) for b in val_labels]
print("Number of entries in the validation data: ", length(val_jets))

length(targets[1, :]) = 98081
Number of entries in the validation data: 

98081

98081-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

Lets examine how one of the jets looks like, each one of the entries is one detected particle's four momentum $(E, p_x, p_y, p_z)$.

However,in order to compute the embeddings, it is necesary to convert the jets to a format that can be used by the framework. The function `data2hyp` allows to convert each detected four momentum to the jet basis, a.k.a $(\tilde p_T, \cos(\theta), \sin(\theta), \tilde y, E_T)$

In [12]:
train_transf_jets = data2basis(train_jets; basis="hyp")
val_transf_jets = data2basis(val_jets; basis="hyp")
println("Transformed jets")

Transformed jets


### The embeddings

Once the jets are converted to the jet basis, it is moment to embed the model using the *Invariant Polynomials*. 

The function `build_ip` allocates efficiently the sparse basis, while the `bip_data` computes the invariant representation of each one of the jets.

In [10]:
f_bip, specs, a_basis = build_ip(order=2, levels=5)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage[:, 2:end]
end

bip_data (generic function with 1 method)

In [7]:
specs

51-element Vector{Int64}:
 0
 1
 1
 1
 1
 1
 1
 2
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 2
 2

In [11]:
train_embedded_jets = bip_data(train_transf_jets)
println("Embedded train jets correclty")
val_embedded_jets = bip_data(val_transf_jets)
println("Embedded test jets correclty")

UndefVarError: UndefVarError: train_transf_jets not defined

In [10]:
length(specs)

51

### Training a classifier model

The embeddings are now created for the dataset. From this point on, the classification itself is absolutelly versatile. For this specific example we will use the out-of-the box classifier `sklearn.linear_model.HistGradientBoostingClassifier` that bines the data and then applies a grandient boosted trees algorithm. 

Now, lets fit a simple model to the data.


In [11]:
using PyCall
@pyimport sklearn.ensemble as sk_ensemble

In [12]:
#GCT = sk_ensemble.HistGradientBoostingClassifier(verbose=false, max_iter=3000, l2_regularization=0.02, learning_rate=0.02, max_depth = 4).fit(train_embedded_jets, train_labels)

# Lest test how we do performance

Now that we understanad the framework, lets see how our model performs on the test set.

In [13]:
test_data_path = "../../../DataLake/raw/test.h5"

"../../../DataLake/raw/test.h5"

In [14]:
test_jets, test_labels = BIPs.read_data("TQ", test_data_path)
test_labels = [reinterpret(Bool, b == 1.0) for b in test_labels]
test_transf_jets = data2basis(test_jets; basis="hyp")
test_embedded_jets = bip_data(test_transf_jets)
print("Embedded test jets correclty")

ErrorException: unable to determine if ../../../DataLake/raw/test.h5 is accessible in the HDF5 format (file may not exist)

In [15]:
test_preds = GCT.score(test_embedded_jets, test_labels)

UndefVarError: UndefVarError: GCT not defined

In [16]:
using DelimitedFiles

In [17]:
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/specs.csv",  specs, ',')

SystemError: SystemError: opening file "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/specs.csv": No such file or directory

In [18]:
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/train_basis.csv",  train_embedded_jets, ',')
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/train_labels.csv",  train_labels, ',')

UndefVarError: UndefVarError: train_embedded_jets not defined

In [19]:
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/val_basis.csv",  val_embedded_jets, ',')
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/val_labels.csv",  val_labels, ',')

UndefVarError: UndefVarError: val_embedded_jets not defined

In [20]:
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/test_basis.csv",  test_embedded_jets, ',')
writedlm( "/home/josemm/MyDocs/personal_BIP/BIPs.jl/foo/storage_basis/test_labels.csv",  test_labels, ',')

UndefVarError: UndefVarError: test_embedded_jets not defined

In [21]:
a_basis.spec[[6,6,1]]

3-element Vector{BIPs.BiPolynomials.Modules.ASpec}:
 BIPs.BiPolynomials.Modules.ASpec(1, 0, 0)
 BIPs.BiPolynomials.Modules.ASpec(1, 0, 0)
 BIPs.BiPolynomials.Modules.ASpec(0, 0, 0)